(lite)

In [ ]:
try:
    import piplite
    await piplite.install(["ipywidgets==7.7.1", "ipyleaflet==0.17.1", "emfs:here_search_widget-0.6.1-py3-none-any.whl"], keep_going=True)
except ImportError:
    pass

## Browse and Derived place taxonomy

`api.browse()` supports a set of places filters around [category](https://developer.here.com/documentation/geocoding-search-api/dev_guide/topics-places/places-category-system-full.html), [cusine](https://developer.here.com/documentation/geocoding-search-api/dev_guide/topics-places/food-types-category-system-full.html), [chain](https://developer.here.com/documentation/geocoding-search-api/dev_guide/topics-places/places-chain-system-full.html) IDs.

For example, to get the nearby places with on of the category IDs `700-7600-0000` (Fueling Station), `700-7600-0116` (Petrol-Gasoline Station) or `700-7600-0444` (Hydrogen Fuel Station), on can send the following Browse request:

In [ ]:
from here_search.api import API, HTTPSession

api = API()
session = HTTPSession()
resp = await api.browse(latitude=52, longitude=13, limit=4, 
                        categories=["700-7600-0000", "700-7600-0116", "700-7600-0444"], session=session)
resp.req.full

In [ ]:
resp.titles

The place filters multi-values allows the support for simple derived ontologies. For example:

In [ ]:
from here_search.entity.place import PlaceTaxonomyExample
from inspect import getsource
from IPython.display import Code

Code(getsource(PlaceTaxonomyExample))

In [ ]:
resp = await api.browse(latitude=52, longitude=13, limit=4, **PlaceTaxonomyExample.taxonomy.pizza.mapping, session=session)
display(resp.req.full)
resp.titles

## Request & Response objects

The `api.get()` method is called by `api.browse()` and the other endpoint methods. It takes a `Request` object and returns a `Response` object.

In [ ]:
from here_search.entity.request import Endpoint, Request, Response
from here_search.api import base_url

req = Request(Endpoint.REVGEOCODE, base_url[Endpoint.REVGEOCODE], {"at": "52,13", "limit": 1})
await api.get(req, session)